<a href="https://colab.research.google.com/github/kikiru328/Bone_Detection/blob/main/%EC%A2%85%EA%B5%AC/tjnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지

In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
import keras


In [ ]:
tf.__version__

'2.4.1'

In [ ]:
%cd /content/drive/MyDrive/project2/2_Modeling/01_model_learning/

[Errno 2] No such file or directory: '/content/drive/MyDrive/project2/2_Modeling/01_model_learning/'
/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## DataFrame 전처리
- 여기서 이미지 사이즈 조절
- 여기서 model이름 설정
- path 조절
- train_test_split

In [ ]:
path_CARPAL = "/content/drive/MyDrive/2차 프로젝트 원본 데이터/crop_img_folders/crop_img2/crop_img/crops/CARPAL"
path_LMCP = "/content/drive/MyDrive/2차 프로젝트 원본 데이터/crop_img_folders/crop_img2/crop_img/crops/LMCP"
path_LPIP = "/content/drive/MyDrive/2차 프로젝트 원본 데이터/crop_img_folders/crop_img2/crop_img/crops/LPIP"
path_MMCP = "/content/drive/MyDrive/2차 프로젝트 원본 데이터/crop_img_folders/crop_img2/crop_img/crops/MMCP"
path_MPIP = "/content/drive/MyDrive/2차 프로젝트 원본 데이터/crop_img_folders/crop_img2/crop_img/crops/MPIP"
path_TMCP = "/content/drive/MyDrive/2차 프로젝트 원본 데이터/crop_img_folders/crop_img2/crop_img/crops/TMCP"
path_IP = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/crop_img_folders/crop_img2/crop_img/crops/IP'


In [ ]:
# pandas로 안열리면 이거 사용
!pip install pickle5

In [ ]:
# pandas로 안열리면 이거 사용
# 케니스엣지 아닌거
import pickle5 as pickle
with open('/content/drive/MyDrive/Colab Notebooks/BA_all_.pkl', "rb") as fh: # 맨 처음 데이터 프레임
  df = pickle.load(fh)

In [ ]:
# 케니스엣지 데이터 프레임

import pickle5 as pickle
with open('/content/drive/MyDrive/Colab Notebooks/sample1103.pkl', "rb") as fh: # 맨 처음 데이터 프레임
  df = pickle.load(fh)

In [ ]:
df.head()

,NO,gender,Group,REGI,DCT_CODE,BD,PD,OD,DIG_AGE,HEIGT,WEIGT,BMI,BONE,BA_1,BA_2,BA_total,BA_zscore,CARPAL,LMCP,MMCP,TMCP,LPIP,MPIP,IP,ST
0,1,0,1568,8110925,1,2013-08-19,2016-08-01,2016-08-01,2.953425,88.9,11.3,14.3,39.0,3.25,3.25,3.250,-1.591672,1_F.jpg,1_F.jpg,1_F.jpg,1_F.jpg,1_F.jpg,1_F.jpg,1_F.jpg,3
1,2,0,440,5988227,2,2008-10-27,2011-10-25,2011-10-25,2.994521,95.9,13.5,14.7,21.0,1.75,1.75,1.750,-1.966593,2_F.jpg,2_F.jpg,2_F.jpg,2_F.jpg,2_F.jpg,2_F.jpg,2_F.jpg,3
2,3,0,1724,8292016,1,2013-08-08,2016-08-11,2016-08-11,3.010959,88.2,13.0,16.8,28.5,2.75,2.00,2.375,-1.810376,3_F.jpg,3_F.jpg,3_F.jpg,3_F.jpg,3_F.jpg,3_F.jpg,3_F.jpg,3
3,4,0,1919,8565274,1,2014-12-28,2018-02-26,2018-02-26,3.167123,88.7,12.9,16.4,27.0,2.75,1.75,2.250,-1.841619,4_F.jpg,4_F.jpg,4_F.jpg,4_F.jpg,4_F.jpg,4_F.jpg,4_F.jpg,3
4,5,0,1264,7743478,1,2010-07-18,2013-10-29,2013-10-29,3.284932,90.8,14.0,17.0,48.0,3.75,4.25,4.000,-1.404211,5_F.jpg,5_F.jpg,5_F.jpg,5_F.jpg,5_F.jpg,5_F.jpg,5_F.jpg,3


## 함수모음
- generator : input과 output을 만들어주는 함수
- plot_it : 학습후 모델 그래프를 만들고 저장해주는 함수
- mae_in_month : 정규화된 값을 학습시키기 때문에 개월로 변경한 mae계산해주는 함수

In [ ]:
ba_mean = df.BONE.mean()
ba_std = df.BONE.std()

In [ ]:
def generator(df,model,n):
    
    gender = np.array(df.gender)
    age = np.array(df.BA_zscore)
    CARPAL_in = []
    LMCP_in = []
    MMCP_in = []
    TMCP_in = []
    LPIP_in = []
    MPIP_in = []
    IP_in = []
    gender_in = []
    age_in = []
    
    error_list = []
    # df에 있는 파일 명을 갖고 이미지를 불러옴
    for i in range(len(df)):
        
        CARPAL_roi = df.loc[i,'CARPAL']
        LMCP_roi = df.loc[i,'LMCP']
        MMCP_roi = df.loc[i,'MMCP']
        TMCP_roi = df.loc[i,'TMCP']
        LPIP_roi = df.loc[i, 'LPIP']
        MPIP_roi = df.loc[i,'MPIP']
        IP_roi = df.loc[i,'IP']
    
        
        img_CARPAL = cv2.imread(path_CARPAL+'/'+CARPAL_roi)
        img_LMCP = cv2.imread(path_LMCP+'/'+LMCP_roi)
        img_MMCP = cv2.imread(path_MMCP+'/'+MMCP_roi)
        img_TMCP = cv2.imread(path_TMCP+'/'+TMCP_roi)
        img_LPIP = cv2.imread(path_LPIP+'/'+LPIP_roi)
        img_MPIP = cv2.imread(path_MPIP+'/'+MPIP_roi)
        img_IP = cv2.imread(path_IP+'/'+IP_roi)
        

        try:
            img_CARPAL = cv2.resize(img_CARPAL, (224,224),cv2.INTER_AREA)
            img_LMCP = cv2.resize(img_LMCP, (75,75),cv2.INTER_AREA)
            img_MMCP = cv2.resize(img_MMCP, (75,75),cv2.INTER_AREA)
            img_TMCP = cv2.resize(img_TMCP, (75,75),cv2.INTER_AREA)
            img_LPIP = cv2.resize(img_LPIP, (75,75),cv2.INTER_AREA)
            img_MPIP = cv2.resize(img_MPIP, (75,75),cv2.INTER_AREA)
            img_IP = cv2.resize(img_IP, (75,75),cv2.INTER_AREA)
            
        except:
            continue
        # 불러온 이미지를 리스트에 넣는다.

        CARPAL_in.append(img_CARPAL)
        LMCP_in.append(img_LMCP)
        MMCP_in.append(img_MMCP)
        TMCP_in.append(img_TMCP)
        LPIP_in.append(img_LPIP)
        MPIP_in.append(img_MPIP)
        IP_in.append(img_IP)
        
        gender_in.append(gender[i])
        age_in.append(age[i])

    # 저장된 이미지를 벡터값으로 저장시킨다.
           
    CARPAL_in = np.array(CARPAL_in)
    LMCP_in = np.array(LMCP_in)
    MMCP_in = np.array(MMCP_in)
    TMCP_in = np.array(TMCP_in)
    LPIP_in = np.array(LPIP_in)
    MPIP_in = np.array(MPIP_in)
    IP_in = np.array(IP_in)

    gender_in = np.array(gender_in)
    age_in = np.array(age_in)
        
    if n==1:
        return age_in
        
    if n==2:
        return [gender_in,CARPAL_in,LMCP_in,MMCP_in,TMCP_in, LPIP_in, MPIP_in, IP_in],age_in
        
    if n==3:
        return [CARPAL_in,LMCP_in,MMCP_in,TMCP_in, LPIP_in, MPIP_in, IP_in,gender_in],age_in    

In [ ]:
def show(img):
    plt.figure(figsize=(15, 10))
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap = "gray")
    plt.show()

In [ ]:
# process시각화 및 손실함수

def plot_it(history):
    '''function to plot training and validation error'''
    fig, ax = plt.subplots( figsize=(20,10))
    ax.plot(history.history['mae_in_months'])
    ax.plot(history.history['val_mae_in_months'])
    plt.title('Model Error')
    plt.ylabel('error')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper right')
    ax.grid(color='black')
    plt.show()
    fig = plt.gcf()
    fig.savefig("/content/drive/MyDrive/Colab Notebooks/"+model_name+".jpg")
    plt.close(fig)

In [ ]:
from tensorflow.keras.metrics import mean_absolute_error

def mae_in_months(x_p, y_p):
    '''function to return mae in months'''
    return mean_absolute_error((ba_std*x_p + ba_mean), (ba_std*y_p + ba_mean)) 

## 모델 생성

In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/TJM1')

In [ ]:
def mae_in_months(x_p, y_p):
    '''function to return mae in months'''
    return mean_absolute_error((ba_std*x_p + ba_mean), (ba_std*y_p + ba_mean))  

In [ ]:
def tjnet3():
  
    
    #원본이미지 모델 (mobilnet)
    #mob_model = tf.keras.applications.MobileNet(input_shape = (bimg_size, bimg_size, 3),
                                                   #include_top = False,
                                                                                  #weights = 'imagenet')
    #mob_model.trainable = True
    #model_2 = Sequential()
    #model_2.add(mob_model)
    #model_2.add(GlobalAveragePooling2D())
    #model_2.add(Dense(1024, activation = 'relu')) # 반으로줄임 

    # CARPALroi 모델 (tj-net)

    i_input = Input(shape = (224,224,3)) # 입력

    #block 1
    conv1 = ReLU()(Conv2D(32,3)(i_input))
    conv2 = ReLU()(Conv2D(32,3)(conv1))
    conv3 = ReLU()(Conv2D(64,3,padding='same')(conv2))
    x = Conv2D(64,1)(conv3) # 합성곱층
    x = BatchNormalization()(x) # 배치 정규화 (이걸 왜쓰냐고? 그야...그것이..낭만이니까)

    se = GlobalAveragePooling2D()(x)
    se = Dense(4,activation="relu")(se)
    se = Dense(64,activation="relu")(se)
    se = Reshape([1,1,64])(se)
    x = Multiply()([conv3,se])

    short = Conv2D(64,(1,1))(conv2)
    short = BatchNormalization()(short)

    x = ReLU()(add([x,short]))

    pool1 = MaxPool2D(pool_size=(2,2))(x)
    #block 2
    conv4 = ReLU()(Conv2D(60,1)(pool1))
    conv5 = ReLU()(Conv2D(192,3)(conv4))
    pool2 = MaxPool2D(pool_size=(2,2))(conv5)
    #block 3
    conv6 = ReLU()(Conv2D(512,3)(pool2))
    pool3 = MaxPool2D(pool_size=(2,2))(conv6)
    #block 4
    conv7 = ReLU()(Conv2D(512,3,padding='same')(pool3)) # 반으로 줄임
    pool4 = MaxPool2D(pool_size=(2,2))(conv7)
    i_output = Flatten()(pool4)

    i_model = Model(inputs = i_input, outputs = i_output)
    
    # LMCP roi 모델 (tj-net)

    i_input1 = Input(shape = (75,75,3))
    
    #block 1
    conv1 = ReLU()(Conv2D(32,3)(i_input1)) 
    conv2 = ReLU()(Conv2D(32,3)(conv1))
    conv3 = ReLU()(Conv2D(64,3,padding='same')(conv2))
    x = Conv2D(64,1)(conv3)
    x = BatchNormalization()(x)

    se = GlobalAveragePooling2D()(x)
    se = Dense(4,activation="relu")(se)
    se = Dense(64,activation="relu")(se)
    se = Reshape([1,1,64])(se)
    x = Multiply()([conv3,se])

    short = Conv2D(64,(1,1))(conv2)
    short = BatchNormalization()(short)

    x = ReLU()(add([x,short]))

    pool1 = MaxPool2D(pool_size=(2,2))(x)
    #block 2
    conv4 = ReLU()(Conv2D(60,1)(pool1))
    conv5 = ReLU()(Conv2D(192,3)(conv4))
    pool2 = MaxPool2D(pool_size=(2,2))(conv5)
    #block 3
    conv6 = ReLU()(Conv2D(256,3)(pool2))
    pool3 = MaxPool2D(pool_size=(2,2))(conv6)
    #block 4
    conv7 = ReLU()(Conv2D(512,3,padding='same')(pool3)) # 반으로 줄임
    pool4 = MaxPool2D(pool_size=(2,2))(conv7)
    i_output1 = Flatten()(pool4)

    i_model1 = Model(inputs = i_input1, outputs = i_output1)

    # MMCP roi 모델 (tj-net)

    i_input2 = Input(shape = (75,75,3))
    
    #block 1
    conv1 = ReLU()(Conv2D(32,3)(i_input2)) 
    conv2 = ReLU()(Conv2D(32,3)(conv1))
    conv3 = ReLU()(Conv2D(64,3,padding='same')(conv2))
    x = Conv2D(64,1)(conv3)
    x = BatchNormalization()(x)

    se = GlobalAveragePooling2D()(x)
    se = Dense(4,activation="relu")(se)
    se = Dense(64,activation="relu")(se)
    se = Reshape([1,1,64])(se)
    x = Multiply()([conv3,se])

    short = Conv2D(64,(1,1))(conv2)
    short = BatchNormalization()(short)

    x = ReLU()(add([x,short]))

    pool1 = MaxPool2D(pool_size=(2,2))(x)
    #block 2
    conv4 = ReLU()(Conv2D(60,1)(pool1))
    conv5 = ReLU()(Conv2D(192,3)(conv4))
    pool2 = MaxPool2D(pool_size=(2,2))(conv5)
    #block 3
    conv6 = ReLU()(Conv2D(256,3)(pool2))
    pool3 = MaxPool2D(pool_size=(2,2))(conv6)
    #block 4
    conv7 = ReLU()(Conv2D(512,3,padding='same')(pool3)) # 반으로 줄임
    pool4 = MaxPool2D(pool_size=(2,2))(conv7)
    i_output2 = Flatten()(pool4)

    i_model2 = Model(inputs = i_input2, outputs = i_output2)

    # TMCP roi 모델 (tj-net)

    i_input3 = Input(shape = (75,75,3))
    
    #block 1
    conv1 = ReLU()(Conv2D(32,3)(i_input3)) 
    conv2 = ReLU()(Conv2D(32,3)(conv1))
    conv3 = ReLU()(Conv2D(64,3,padding='same')(conv2))
    x = Conv2D(64,1)(conv3)
    x = BatchNormalization()(x)

    se = GlobalAveragePooling2D()(x)
    se = Dense(4,activation="relu")(se)
    se = Dense(64,activation="relu")(se)
    se = Reshape([1,1,64])(se)
    x = Multiply()([conv3,se])

    short = Conv2D(64,(1,1))(conv2)
    short = BatchNormalization()(short)

    x = ReLU()(add([x,short]))

    pool1 = MaxPool2D(pool_size=(2,2))(x)
    #block 2
    conv4 = ReLU()(Conv2D(60,1)(pool1))
    conv5 = ReLU()(Conv2D(192,3)(conv4))
    pool2 = MaxPool2D(pool_size=(2,2))(conv5)
    #block 3
    conv6 = ReLU()(Conv2D(256,3)(pool2))
    pool3 = MaxPool2D(pool_size=(2,2))(conv6)
    #block 4
    conv7 = ReLU()(Conv2D(512,3,padding='same')(pool3)) # 반으로 줄임
    pool4 = MaxPool2D(pool_size=(2,2))(conv7)
    i_output3 = Flatten()(pool4)

    i_model3 = Model(inputs = i_input3, outputs = i_output3)

    # LPIP roi 모델 (tj-net)

    i_input4 = Input(shape = (75,75,3))
    
    #block 1
    conv1 = ReLU()(Conv2D(32,3)(i_input4)) 
    conv2 = ReLU()(Conv2D(32,3)(conv1))
    conv3 = ReLU()(Conv2D(64,3,padding='same')(conv2))
    x = Conv2D(64,1)(conv3)
    x = BatchNormalization()(x)

    se = GlobalAveragePooling2D()(x)
    se = Dense(4,activation="relu")(se)
    se = Dense(64,activation="relu")(se)
    se = Reshape([1,1,64])(se)
    x = Multiply()([conv3,se])

    short = Conv2D(64,(1,1))(conv2)
    short = BatchNormalization()(short)

    x = ReLU()(add([x,short]))

    pool1 = MaxPool2D(pool_size=(2,2))(x)
    #block 2
    conv4 = ReLU()(Conv2D(60,1)(pool1))
    conv5 = ReLU()(Conv2D(192,3)(conv4))
    pool2 = MaxPool2D(pool_size=(2,2))(conv5)
    #block 3
    conv6 = ReLU()(Conv2D(256,3)(pool2))
    pool3 = MaxPool2D(pool_size=(2,2))(conv6)
    #block 4
    conv7 = ReLU()(Conv2D(512,3,padding='same')(pool3)) # 반으로 줄임
    pool4 = MaxPool2D(pool_size=(2,2))(conv7)
    i_output4 = Flatten()(pool4)

    i_model4 = Model(inputs = i_input4, outputs = i_output4)

    
    # MPIP roi 모델 (tj-net)

    i_input5 = Input(shape = (75,75,3))
    
    #block 1
    conv1 = ReLU()(Conv2D(32,3)(i_input5))
    conv2 = ReLU()(Conv2D(32,3)(conv1))
    conv3 = ReLU()(Conv2D(64,3,padding='same')(conv2))
    x = Conv2D(64,1)(conv3)
    x = BatchNormalization()(x)

    se = GlobalAveragePooling2D()(x)
    se = Dense(4,activation="relu")(se)    
    se = Dense(64,activation="relu")(se)   
    se = Reshape([1,1,64])(se)
    x = Multiply()([conv3,se])

    short = Conv2D(64,(1,1))(conv2)
    short = BatchNormalization()(short)

    x = ReLU()(add([x,short]))

    pool1 = MaxPool2D(pool_size=(2,2))(x)
    #block 2
    conv4 = ReLU()(Conv2D(60,1)(pool1))
    conv5 = ReLU()(Conv2D(192,3)(conv4))
    pool2 = MaxPool2D(pool_size=(2,2))(conv5)
    #block 3
    conv6 = ReLU()(Conv2D(256,3)(pool2))
    pool3 = MaxPool2D(pool_size=(2,2))(conv6)
    #block 4
    conv7 = ReLU()(Conv2D(512,3,padding='same')(pool3)) #반으로 줄임
    pool4 = MaxPool2D(pool_size=(2,2))(conv7)
    i_output5 = Flatten()(pool4)

    i_model5 = Model(inputs = i_input5, outputs = i_output5)

    
    # IP roi 모델 (tj-net)

    i_input6 = Input(shape = (75,75,3))
    
    #block 1
    conv1 = ReLU()(Conv2D(32,3)(i_input6))
    conv2 = ReLU()(Conv2D(32,3)(conv1))
    conv3 = ReLU()(Conv2D(64,3,padding='same')(conv2))
    x = Conv2D(64,1)(conv3)
    x = BatchNormalization()(x)

    se = GlobalAveragePooling2D()(x)
    se = Dense(4,activation="relu")(se)
    se = Dense(64,activation="relu")(se)
    se = Reshape([1,1,64])(se)
    x = Multiply()([conv3,se])

    short = Conv2D(64,(1,1))(conv2)
    short = BatchNormalization()(short)

    x = ReLU()(add([x,short]))

    pool1 = MaxPool2D(pool_size=(2,2))(x)
    #block 2
    conv4 = ReLU()(Conv2D(60,1)(pool1))
    conv5 = ReLU()(Conv2D(192,3)(conv4))
    pool2 = MaxPool2D(pool_size=(2,2))(conv5)
    #block 3
    conv6 = ReLU()(Conv2D(256,3)(pool2))
    pool3 = MaxPool2D(pool_size=(2,2))(conv6)
    #block 4
    conv7 = ReLU()(Conv2D(512,3,padding='same')(pool3)) # 이부분을 1024에서 512 로변경 
    pool4 = MaxPool2D(pool_size=(2,2))(conv7)
    i_output6 = Flatten()(pool4)

    i_model6 = Model(inputs = i_input6, outputs = i_output6)

    # gender 모델
    g_model = gender()

    #concat및 마무리
    con1 = concatenate([i_model.output,i_model1.output,i_model2.output,i_model3.output])
    dense1 = ReLU()(Dense(1024)(con1)) # dense1,2,3을 다 반으로 변경
    con2 = concatenate([dense1,g_model.output])
    dense2 = ReLU()(Dense(512)(con2))
    dense3 = ReLU()(Dense(256)(dense2))
    model_out = Dense(1,activation = "linear")(dense3)
    model = Model([i_model.input,i_model1.input,i_model2.input,i_model3.input,i_model4.input,i_model5.input,i_model6.input,g_model.input],model_out)
    return model


In [ ]:
def gender(act='relu'):
    g_input = Input(shape = (1,))
    g_output = Dense(64,activation = act)(g_input)
    
    g_model = Model(inputs = g_input,outputs = g_output)
    return g_model

#**모델 생성(만들때 마다 이름 바꿔주셈)**

In [ ]:
model_name = "tjnet10" # 이거 꼭 바꿔 다시 할때도 바꿔 잘못 만든거면 만들었던거 지워!
model_weight = model_name +"_weight.h5"

model = tjnet3()
model.save("/content/drive/MyDrive/Colab Notebooks/"+model_name+".h5") # 알아서 경로 잡아

model.compile(loss ='mse', optimizer= 'nadam', metrics = [mae_in_months] )

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
model.input

[<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_9')>,
 <KerasTensor: shape=(None, 75, 75, 3) dtype=float32 (created by layer 'input_10')>,
 <KerasTensor: shape=(None, 75, 75, 3) dtype=float32 (created by layer 'input_11')>,
 <KerasTensor: shape=(None, 75, 75, 3) dtype=float32 (created by layer 'input_12')>,
 <KerasTensor: shape=(None, 75, 75, 3) dtype=float32 (created by layer 'input_13')>,
 <KerasTensor: shape=(None, 75, 75, 3) dtype=float32 (created by layer 'input_14')>,
 <KerasTensor: shape=(None, 75, 75, 3) dtype=float32 (created by layer 'input_15')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_16')>]

In [ ]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 75, 75, 3)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 75, 75, 3)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 75, 75, 3)]  0                                            
____________________________________________________________________________________________

## train_val_set 만들기

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

# train_df, val_df = train_test_split(df, test_size = 0.2, random_state = 0)

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.3, random_state = 22)

# ST를 기준으로 일정 비율로 데이터셋을 나눔
for train_index, test_index in split.split(df, df['ST']):
    train_df = df.loc[train_index]
    val_df = df.loc[test_index]

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

train_X,train_y = generator(train_df,model,3) # 7:3비율로 나눔, 데이터 프레임에 있는 roi파일 이름과 이미지를 맞춰 셋을 나눔
val_X,val_y = generator(val_df,model,3)

In [ ]:
len(train_df), len(val_df)

(865, 371)

In [ ]:
len(train_y),len(val_y)

(865, 371)

## 초반에 loss 심한거 뺄려고 한번만 먼저 돌리기

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4288300965539651157
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16185556992
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13533165694376665704
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [ ]:
# model.load_weights("temp/"+model_weight)

model.fit(train_X,train_y,validation_data = (val_X,val_y),epochs = 1,batch_size=4)

216/216 [==============================] - 30s 103ms/step - loss: 141.6041 - mae_in_months: 62.0321 - val_loss: 0.0809 - val_mae_in_months: 11.5675


## 모델학습

In [ ]:
# 모델 fit

epochs = 100                                                 
#early stopping
early_stopping = [EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience= 5,
                              verbose=0, mode='auto'),
                  ModelCheckpoint(filepath='/content/drive/MyDrive/Colab Notebooks/tjnet5.h5',
                                  monitor = 'val_loss',
                                  save_best_only = True)] # 이거도 이름 바꿔줘 


# mc = ModelCheckpoint("temp/"+model_weight, monitor='val_loss', mode='min', save_best_only=True)
rLR = ReduceLROnPlateau(
    monitor='val_loss',  # 검증 손실을 기준으로 callback이 호출됩니다
    factor=0.5,          # callback 호출시 학습률을 1/2로 줄입니다
    patience=5,         # epoch 5 동안 개선되지 않으면 callback이 호출됩니다
)

callbacks = [early_stopping,rLR]


#fit model
history = model.fit(train_X,train_y,
                    validation_data = (val_X,val_y),
                    epochs = epochs,
                    callbacks= callbacks,
                    batch_size=4)
plot_it(history)

Epoch 1/100
216/216 [==============================] - 21s 98ms/step - loss: 0.0548 - mae_in_months: 8.9305 - val_loss: 0.0768 - val_mae_in_months: 10.5232


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
216/216 [==============================] - 21s 99ms/step - loss: 0.0450 - mae_in_months: 7.7817 - val_loss: 0.0264 - val_mae_in_months: 5.9154
Epoch 3/100
216/216 [==============================] - 21s 99ms/step - loss: 0.0243 - mae_in_months: 5.7021 - val_loss: 0.0318 - val_mae_in_months: 6.9677
Epoch 4/100
216/216 [==============================] - 21s 99ms/step - loss: 0.0203 - mae_in_months: 5.2430 - val_loss: 0.0212 - val_mae_in_months: 5.4950
Epoch 5/100
193/216 [=========================>....] - ETA: 2s - loss: 0.0120 - mae_in_months: 4.0818

In [ ]:
history

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs,acc,'bo',label='Training acc')
plt.plot(epochs,val_acc,'b',label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Trainindg and validation loss')
plt.legend()
plt.show()

KeyError: ignored

## 평가 및 저장

In [ ]:
pred = model.predict(val_X)*ba_std+ba_mean
mse, mae = model.evaluate(val_X,val_y)

from sklearn.metrics import r2_score
# r2 = r2_score(val_df.boneage,pred)

print(f"mae_year : {mae}, rmse : {mse}" )

12/12 [==============================] - 1s 72ms/step - loss: 0.0167 - mae_in_months: 4.9390
mae_year : 4.9390058517456055, rmse : 0.016723008826375008


In [ ]:
from sklearn.metrics import mean_absolute_error
v = val_y*ba_std+ba_mean
mean_absolute_error(np.round(pred),v)/12,mean_absolute_error(pred,v)/12

(0.4112631639640769, 0.4115838367486278)

In [ ]:
import numpy as np
import tensorflow.keras as tf

model = tf.models.load_model('/content/drive/MyDrive/Colab Notebooks/tjnet4.h5')

CARPAL = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/crop_img_folders/CARPAL/100_F_cp.jpg'

y_predict = model.predict(x_test)

label = labels[y_predict[0].argmax()]
confidence = y_predict[0][y_predict[0].argmax()]